#**Additional NOTE: THIS WAS A PAIN!!!!!!!!!!** _also fun_

In [1]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove.6B.zip

--2025-05-29 11:57:34--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2025-05-29 11:57:34--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2025-05-29 11:57:34--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

#Tokenising English Sentences using GLOVE

In [2]:
import torch

In [3]:
def load_glove_embeddings(filepath, embedding_dim=100):
    embeddings = {}
    with open(filepath, 'r', encoding='utf8') as f:
        for line in f:
            parts = line.strip().split()
            word = parts[0]
            vector = torch.tensor([float(x) for x in parts[1:]], dtype=torch.float)
            if len(vector) == embedding_dim:
                embeddings[word] = vector
    return embeddings

glove_path = "glove.6B.100d.txt" # We are using this glove embedding of dim =100
glove = load_glove_embeddings(glove_path, embedding_dim=100)


import string

def simple_tokenize(sentence):
    # Lowercase + split at whitespace
    tokens = sentence.lower().strip().split()
    # Remove punctuation from each token
    table = str.maketrans('', '', string.punctuation)
    stripped = [w.translate(table) for w in tokens]
    # Filter out any empty tokens (e.g., "" from "--" or "!!")
    return [w for w in stripped if w]

from nltk.tokenize import word_tokenize

def encode_glove(sentence, max_len=60, embedding_dim=100):
    tokens = simple_tokenize(sentence.lower())[:max_len]
    tensor = torch.zeros(max_len, embedding_dim)

    for i, token in enumerate(tokens):
        if token in glove:
            tensor[i] = glove[token]
        else:
            tensor[i] = torch.zeros(embedding_dim)  # or random vector
    return tensor

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import numpy as np
from transformers import AutoTokenizer
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

Reading the Translation data using Pandas

In [5]:
import pandas as pd
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
df = pd.read_csv("/content/translation_engtohin_train.csv").dropna()
source_sentences = df["source"].tolist()
target_sentences = df["target"].tolist()

# limiting source sentences because Colab keeps throwing Usage limit.
# note: decreasing this did increase the errors to a good extent.
num_sentences = 55000
source_sentences = source_sentences[:num_sentences]
target_sentences = target_sentences[:num_sentences]

df = pd.read_csv("/content/translation_engtohin_valid.csv").dropna()
source_sentences_val = df["source"].tolist()
target_sentences_val = df["target"].tolist()
print(device)
print(len(source_sentences))

cuda
55000


#Tokenising Hindi Sentences using IndicBert

*The below code can be used to analyse the length of sentences present in the dataset which is helpful to get an insight defining maximum length of source and target that you are gonna use to train the model*

In [6]:
tokenizer_hin = AutoTokenizer.from_pretrained("ai4bharat/IndicBERTv2-MLM-only")
hin_vocab_size = tokenizer_hin.vocab_size
print(hin_vocab_size, "hin_vocab_size")

import re

def simple_tokenize_en(text):
    return re.findall(r'\w+', text)  # Split on words (alphanumeric)

# Count for English
long_en_count = sum(len(simple_tokenize_en(sent)) > 60 for sent in source_sentences)

# Count for Marathi using IndicBERT tokenizer
long_mr_count = sum(
    len(tokenizer_hin(sent, truncation=False, padding=False)["input_ids"]) > 70
    for sent in target_sentences
)

print(f"Number of English sentences with >60 tokens: {long_en_count}")
print(f"Number of Hindi sentences with >70 tokens: {long_mr_count}")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/51.0 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.75M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

250000 hin_vocab_size
Number of English sentences with >60 tokens: 457
Number of Hindi sentences with >70 tokens: 1070


In [7]:
def encode_hin(sentence, max_len=70):
    token_ids = tokenizer_hin(sentence, padding='max_length', truncation=True, max_length=max_len, return_tensors='pt')["input_ids"].squeeze(0)
    return token_ids

In [8]:
from torch.utils.data import DataLoader, Dataset
import torch.nn.functional as F

class TranslationDataset(Dataset):
    def __init__(self, source_sentences, target_sentences, max_src_len, max_tgt_len):
        self.source_sentences = source_sentences
        self.target_sentences = target_sentences
        self.max_src_len = max_src_len
        self.max_tgt_len = max_tgt_len

    def __len__(self):
        return len(self.source_sentences)

    def __getitem__(self, idx):
        src_vec = encode_glove(self.source_sentences[idx], self.max_src_len)  # [max_src_len, 100]
        tgt_ids = encode_hin(self.target_sentences[idx], self.max_tgt_len)  # [max_tgt_len]
        return src_vec, tgt_ids

def collate_fn(batch):
    src_batch, tgt_batch = zip(*batch)

    src_batch = torch.stack(src_batch)  # [B, max_src_len, 100]
    tgt_batch = torch.stack(tgt_batch)  # [B, max_tgt_len]
    return src_batch, tgt_batch

| Component               | Purpose                                               |
| ----------------------- | ----------------------------------------------------- |
| `simple_tokenize`       | Clean and tokenize English sentences manually         |
| `load_glove_embeddings` | Load GloVe vectors into a dictionary                  |
| `encode_glove`          | Convert English text to GloVe vector matrix           |
| `AutoTokenizer`         | Pre-trained tokenizer for Hindi                       |
| `encode_hin`            | Convert Hindi sentence into token IDs                 |
| `TranslationDataset`    | Dataset to serve sentence pairs for training          |
| `collate_fn`            | Create uniform batches from variable-length sentences |


In [9]:
max_src_len = 10 #decide yourself
max_tgt_len = 10 #decide yourself
batch_size = 64 #decide yourself (GPU ran out of memory for batch size of 128, and lowering to 32 or 16 SIGNIFICANTLY increased training time, it was pain)

# Dataset and loaders
train_dataset = TranslationDataset(source_sentences, target_sentences, max_src_len, max_tgt_len)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, collate_fn=collate_fn)

val_dataset = TranslationDataset(source_sentences_val, target_sentences_val, max_src_len, max_tgt_len)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

In [10]:
print("Source dataset length English",train_dataset[0][0].shape)
print("Source dataset length Hindi",train_dataset[0][1].shape)
print(train_dataset[0][0].shape)
print(len(train_dataset))

Source dataset length English torch.Size([10, 100])
Source dataset length Hindi torch.Size([10])
torch.Size([10, 100])
55000


*Your task is to build a sequence-to-sequence model using a RNN  to perform machine translation. The dataset is already preprocessed and tokenized, and embeddings for both source and target languages are provided. Implement the encoder-decoder architecture using RNNs, train your model on the training split, validate it on the validation split, and finally evaluate it on the test set using BLEU score.*

In [ ]:
# write your code here

## Using a Many-to-Many Architecture here
with 3 hidden layers in encoder and decoder each.

In [11]:
import pandas as pd
'''

class Encoder(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        # num_layers parameter specifies the number of recurrent layers
        self.rnn = nn.RNN(input_dim, hidden_dim, num_layers=num_layers, batch_first=True)

    def forward(self, src):
        # src: [batch_size, max_src_len, input_dim]
        # output: [batch_size, max_src_len, hidden_dim]
        # hidden: [num_layers, batch_size, hidden_dim]
        output, hidden = self.rnn(src)
        return hidden

class Decoder(nn.Module):
    def __init__(self, output_dim, hidden_dim, num_layers):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.num_layers = num_layers
        # Embedding for target tokens, maps token IDs to hidden_dim vec
        self.embedding = nn.Embedding(output_dim, hidden_dim)
        self.rnn = nn.RNN(hidden_dim, hidden_dim, num_layers=num_layers, batch_first=True)
        self.fc_out = nn.Linear(hidden_dim, output_dim)

    def forward(self, input, hidden):
        # input: [batch_size] (a batch of single token IDs at a time step)
        # hidden: [num_layers, batch_size, hidden_dim] (initial hidden state from encoder or previous decoder step)

        # Add a sequence dimension to the input for RNN
        input = input.unsqueeze(1) # [batch_size, 1]

        # Embed the input token
        embedded = self.embedding(input) # [batch_size, 1, hidden_dim]

        # Pass embedded input and previous hidden state through RNN
        # output: [batch_size, 1, hidden_dim]
        # hidden: [num_layers, batch_size, hidden_dim] (updated hidden state for all layers)
        output, hidden = self.rnn(embedded, hidden)

        # Predict the next token (need to remove the sequence dimension for the linear layer)
        prediction = self.fc_out(output.squeeze(1)) # [batch_size, output_dim]

        return prediction, hidden
'''

'\n\nclass Encoder(nn.Module):\n    def __init__(self, input_dim, hidden_dim, num_layers):\n        super().__init__()\n        self.hidden_dim = hidden_dim\n        self.num_layers = num_layers\n        # num_layers parameter specifies the number of recurrent layers\n        self.rnn = nn.RNN(input_dim, hidden_dim, num_layers=num_layers, batch_first=True)\n\n    def forward(self, src):\n        # src: [batch_size, max_src_len, input_dim]\n        # output: [batch_size, max_src_len, hidden_dim]\n        # hidden: [num_layers, batch_size, hidden_dim]\n        output, hidden = self.rnn(src)\n        return hidden\n\nclass Decoder(nn.Module):\n    def __init__(self, output_dim, hidden_dim, num_layers):\n        super().__init__()\n        self.hidden_dim = hidden_dim\n        self.output_dim = output_dim\n        self.num_layers = num_layers\n        # Embedding for target tokens, maps token IDs to hidden_dim vec\n        self.embedding = nn.Embedding(output_dim, hidden_dim)\n        self

In [ ]:
'''
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device

    def forward(self, src, trg):
        # src: [batch_size, max_src_len, input_dim]
        # trg: [batch_size, max_tgt_len] (contains token IDs)

        batch_size = trg.shape[0]
        max_tgt_len = trg.shape[1]
        trg_vocab_size = self.decoder.output_dim

        # Tensor to store decoder outputs
        outputs = torch.zeros(batch_size, max_tgt_len, trg_vocab_size).to(self.device)

        # Encode source sentence
        encoder_hidden = self.encoder(src)

        # First input to the decoder is the <sos> token
        # We need to assume a special token for the start of sequence.
        # For IndicBERT, the <sos> token is the CLS token (usually 101)
        # and the <eos> token is the SEP token (usually 102).
        # We will use the CLS token as the initial input to the decoder.
        input = trg[:, 0] # Use the first token of the target sequence (should be <sos>)

        for t in range(1, max_tgt_len): # Start from 1 because the first token was the input
            # Pass input token and hidden state to decoder
            output, encoder_hidden = self.decoder(input, encoder_hidden)

            # Store the output prediction
            outputs[:, t, :] = output

            # Decide next input token
            # Use teacher forcing (feed ground truth target token as next input)
            input = trg[:, t]

        return outputs
'''
'''
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device

        # Ensure encoder and decoder have the same number of layers for hidden state compatibility
        assert encoder.num_layers == decoder.num_layers, \
            "Encoder and decoder must have equal number of layers!"

    def forward(self, src, trg):
        # src: [batch_size, max_src_len, input_dim]
        # trg: [batch_size, max_tgt_len] (contains token IDs)

        batch_size = trg.shape[0]
        max_tgt_len = trg.shape[1]
        trg_vocab_size = self.decoder.output_dim

        # Tensor to store decoder outputs (logits for each token at each time step)
        outputs = torch.zeros(batch_size, max_tgt_len, trg_vocab_size).to(self.device)

        # Encode source sentence to get the final hidden state for all layers
        # encoder_hidden: [num_layers, batch_size, hidden_dim]
        encoder_hidden = self.encoder(src)

        # First input to the decoder is the <sos> token (CLS token for IndicBERT)
        input = trg[:, 0] # [batch_size]

        # Iterate through the target sequence, predicting one token at a time
        # Start from 1 because the first token (trg[:, 0]) was the initial input
        for t in range(1, max_tgt_len):
            # Pass input token and hidden state to decoder
            # output: [batch_size, trg_vocab_size] (logits for next token)
            # encoder_hidden: [num_layers, batch_size, hidden_dim] (updated hidden state)
            output, encoder_hidden = self.decoder(input, encoder_hidden)

            # Store the output prediction
            outputs[:, t, :] = output

            # Decide next input token using teacher forcing (feed ground truth target token)
            input = trg[:, t]

        return outputs

'''

In [18]:
"""
# trial - Attention Module
class Attention(nn.Module):
    def __init__(self, hidden_dim):
        super().__init__()
        # Linear layer to combine decoder hidden state and encoder outputs for 'importance' calculation
        # hidden_dim * 2 because we concatenate decoder_hidden_last_layer and encoder_output
        self.attn = nn.Linear(hidden_dim * 2, hidden_dim)
        # Linear layer to project the importance to a scalar score
        self.v = nn.Linear(hidden_dim, 1, bias=False)

    def forward(self, decoder_hidden_last_layer, encoder_outputs):
        # decoder_hidden_last_layer: [batch_size, hidden_dim] (last layer hidden state of decoder)
        # encoder_outputs: [batch_size, max_src_len, hidden_dim] (all encoder outputs from encoder)

        batch_size = encoder_outputs.shape[0]
        max_src_len = encoder_outputs.shape[1]

        # Repeat the decoder's last layer hidden state
        # This prepares it for concatenation with each encoder output
        # decoder_hidden_last_layer_repeated: [batch_size, max_src_len, hidden_dim]
        decoder_hidden_last_layer_repeated = decoder_hidden_last_layer.unsqueeze(1).repeat(1, max_src_len, 1)

        # Concatenate the repeated decoder hidden state with all encoder outputs
        # energy_input: [batch_size, max_src_len, hidden_dim * 2]
        energy_input = torch.cat((decoder_hidden_last_layer_repeated, encoder_outputs), dim=2) # energy is basically the importance

        # Apply the attention scoring function (tanh and then linear projection)
        # energy: [batch_size, max_src_len, hidden_dim]
        energy = torch.tanh(self.attn(energy_input))

        # Project the energy to a single scalar score for each source token
        # attention_scores: [batch_size, max_src_len]
        attention_scores = self.v(energy).squeeze(2)

        # Apply softmax to get attention weights, ensuring they sum to 1 across the source sequence
        # attention_weights: [batch_size, max_src_len]
        return F.softmax(attention_scores, dim=1)

'''
# Encoder, Decoder, and Seq2Seq Classes with Attention
class Encoder(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        # num_layers parameter specifies the number of recurrent layers
        self.rnn = nn.RNN(input_dim, hidden_dim, num_layers=num_layers, batch_first=True)

    def forward(self, src):
        # src: [batch_size, max_src_len, input_dim]
        # output: [batch_size, max_src_len, hidden_dim] (all hidden states of the top layer)
        # hidden: [num_layers, batch_size, hidden_dim] (final hidden state for all layers)
        output, hidden = self.rnn(src)
        return output, hidden # Return both outputs and final hidden state


class Decoder(nn.Module):
    def __init__(self, output_dim, hidden_dim, num_layers, attention):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.num_layers = num_layers
        self.attention = attention # Attention module instance

        # Embedding for target language tokens, maps token IDs to hidden_dim vectors
        self.embedding = nn.Embedding(output_dim, hidden_dim)
        # RNN for the decoder
        self.rnn = nn.RNN(hidden_dim, hidden_dim, num_layers=num_layers, batch_first=True)
        # Final output layer: takes concatenated RNN output and context vector
        # hidden_dim (from rnn_output) + hidden_dim (from context_vector)
        self.fc_out = nn.Linear(hidden_dim * 2, output_dim)

    def forward(self, input, hidden, encoder_outputs):
        # input: [batch_size] (a batch of single token IDs at a time step)
        # hidden: [num_layers, batch_size, hidden_dim] (initial hidden state from encoder or previous decoder step)
        # encoder_outputs: [batch_size, max_src_len, hidden_dim] (all encoder outputs)

        # Add a sequence dimension to the input for RNN (RNN expects 3D input)
        input = input.unsqueeze(1) # [batch_size, 1]

        # Embed the input token
        embedded = self.embedding(input) # [batch_size, 1, hidden_dim]

        # Pass embedded input and previous hidden state through RNN
        # rnn_output: [batch_size, 1, hidden_dim] (output of the top RNN layer for the current time step)
        # hidden: [num_layers, batch_size, hidden_dim] (updated hidden state for all layers)
        rnn_output, hidden = self.rnn(embedded, hidden)

        # Get the hidden state of the last layer of the decoder for attention calculation
        # decoder_hidden_last_layer: [batch_size, hidden_dim]
        decoder_hidden_last_layer = hidden[-1]

        # Calculate attention weights based on decoder's current hidden state and all encoder outputs
        # attention_weights: [batch_size, max_src_len]
        attention_weights = self.attention(decoder_hidden_last_layer, encoder_outputs)

        # Apply attention weights to encoder outputs to get the context vector
        # context_vector: [batch_size, hidden_dim]
        # torch.bmm performs batch matrix-matrix product: (B, 1, L) x (B, L, H) -> (B, 1, H)
        context_vector = torch.bmm(attention_weights.unsqueeze(1), encoder_outputs).squeeze(1)

        # Concatenate the decoder RNN output (squeezed to remove sequence dim) with the context vector
        # combined_input_for_fc: [batch_size, hidden_dim * 2]
        combined_input_for_fc = torch.cat((rnn_output.squeeze(1), context_vector), dim=1)

        # Predict the next token using the combined information
        prediction = self.fc_out(combined_input_for_fc) # [batch_size, output_dim]

        return prediction, hidden

class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device

        # Ensure encoder and decoder have the same number of layers for hidden state compatibility
        assert encoder.num_layers == decoder.num_layers, \
            "Encoder and decoder must have equal number of layers!"

    def forward(self, src, trg):
        # src: [batch_size, max_src_len, input_dim]
        # trg: [batch_size, max_tgt_len] (contains token IDs)

        batch_size = trg.shape[0]
        max_tgt_len = trg.shape[1]
        trg_vocab_size = self.decoder.output_dim

        # Tensor to store decoder outputs (logits for each token at each time step)
        outputs = torch.zeros(batch_size, max_tgt_len, trg_vocab_size).to(self.device)

        # Encode source sentence to get all hidden states (for attention)
        # and the final hidden state (to initialize decoder)
        # encoder_outputs: [batch_size, max_src_len, hidden_dim]
        # encoder_hidden: [num_layers, batch_size, hidden_dim]
        encoder_outputs, encoder_hidden = self.encoder(src)

        # First input to the decoder is the <sos> token (CLS token for IndicBERT)
        input = trg[:, 0] # [batch_size]

        # Iterate through the target sequence, predicting one token at a time
        # Start from 1 because the first token (trg[:, 0]) was the initial input
        for t in range(1, max_tgt_len):
            # Pass input token, hidden state, and all encoder outputs to decoder
            # output: [batch_size, trg_vocab_size] (logits for next token)
            # encoder_hidden: [num_layers, batch_size, hidden_dim] (updated hidden state)
            output, encoder_hidden = self.decoder(input, encoder_hidden, encoder_outputs)

            # Store the output prediction
            outputs[:, t, :] = output

            # Decide next input token using teacher forcing (feed ground truth target token)
            input = trg[:, t]

        return outputs
'''


## Encoder using GRU
'''
class EncoderGRU(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        # Use nn.GRU instead of nn.RNN or nn.LSTM
        self.rnn = nn.GRU(input_dim, hidden_dim, num_layers=num_layers, batch_first=True)

    def forward(self, src):
        # src: [batch_size, max_src_len, input_dim]
        # output: [batch_size, max_src_len, hidden_dim] (all hidden states of the top layer)
        # hidden: [num_layers, batch_size, hidden_dim] (final hidden state for all layers)
        output, hidden = self.rnn(src)
        # Return all outputs (for attention) and the final hidden state
        return output, hidden
'''

## Encoder using Bidirectional GRU (Refined)

class EncoderBiGRU(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        # Use nn.GRU with bidirectional=True
        self.rnn = nn.GRU(input_dim, hidden_dim, num_layers=num_layers, bidirectional=True, batch_first=True)

        # We will need a way to initialize the decoder's hidden state
        # which is typically of size [num_layers, batch_size, hidden_dim].
        # The bidirectional encoder's final hidden state is [num_layers * 2, batch_size, hidden_dim].
        # A common approach is to concatenate the forward and backward final states
        # and pass them through a linear layer to match the decoder's hidden size.
        # Here, we will concatenate the final hidden states for each layer.
        self.fc_hidden = nn.Linear(hidden_dim * 2, hidden_dim)
        # Initialize this linear layer with appropriate weights (optional but can help)
        # nn.init.xavier_uniform_(self.fc_hidden.weight) # Example initialization

    def forward(self, src):
        # src: [batch_size, max_src_len, input_dim]
        # output: [batch_size, max_src_len, hidden_dim * 2]
        # hidden: [num_layers * 2, batch_size, hidden_dim]
        output, hidden = self.rnn(src)

        # Combine forward and backward final hidden states for each layer
        # hidden_forward: [num_layers, batch_size, hidden_dim]
        # hidden_backward: [num_layers, batch_size, hidden_dim]
        hidden_forward = hidden[0:self.num_layers]
        hidden_backward = hidden[self.num_layers:]

        # Concatenate forward and backward states for each layer
        # combined_hidden: [num_layers, batch_size, hidden_dim * 2]
        combined_hidden = torch.cat((hidden_forward, hidden_backward), dim=2)

        # Project the combined hidden state to the decoder's expected hidden dimension
        # final_encoder_hidden: [num_layers, batch_size, hidden_dim]
        final_encoder_hidden = torch.relu(self.fc_hidden(combined_hidden)) # Use ReLU or Tanh activation

        # Return all time-step outputs (for attention) and the processed final hidden state (for decoder initialization)
        return output, final_encoder_hidden


## Decoder using GRU with Attention

class DecoderGRU(nn.Module):
    def __init__(self, output_dim, hidden_dim, num_layers, attention):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.num_layers = num_layers
        self.attention = attention # Attention module instance

        # Embedding for target language tokens
        self.embedding = nn.Embedding(output_dim, hidden_dim)
        # Use nn.GRU instead of nn.RNN or nn.LSTM
        # Input dimension is hidden_dim (from embedding) + hidden_dim (from context vector)
        self.rnn = nn.GRU(hidden_dim * 2, hidden_dim, num_layers=num_layers, batch_first=True)
        # Final output layer: takes concatenated RNN output and context vector
        self.fc_out = nn.Linear(hidden_dim * 2, output_dim)

    def forward(self, input, hidden, encoder_outputs):
        # input: [batch_size] (a batch of single token IDs at a time step)
        # hidden: [num_layers, batch_size, hidden_dim] (initial hidden state from encoder or previous decoder step)
        # encoder_outputs: [batch_size, max_src_len, hidden_dim] (all encoder outputs)

        # Add a sequence dimension to the input for GRU
        input = input.unsqueeze(1) # [batch_size, 1]

        # Embed the input token
        embedded = self.embedding(input) # [batch_size, 1, hidden_dim]

        # Get the hidden state of the last layer of the decoder for attention calculation
        # decoder_hidden_last_layer: [batch_size, hidden_dim]
        decoder_hidden_last_layer = hidden[-1]

        # Calculate attention weights based on decoder's current hidden state and all encoder outputs
        # attention_weights: [batch_size, max_src_len]
        attention_weights = self.attention(decoder_hidden_last_layer, encoder_outputs)

        # Apply attention weights to encoder outputs to get the context vector
        # context_vector: [batch_size, hidden_dim]
        context_vector = torch.bmm(attention_weights.unsqueeze(1), encoder_outputs).squeeze(1)

        # Concatenate the embedded input with the context vector for the GRU input
        # embedded_with_context: [batch_size, 1, hidden_dim * 2]
        embedded_with_context = torch.cat((embedded, context_vector.unsqueeze(1)), dim=2)

        # Pass combined input and previous hidden state through GRU
        # rnn_output: [batch_size, 1, hidden_dim] (output of the top GRU layer)
        # hidden: [num_layers, batch_size, hidden_dim] (updated hidden state)
        rnn_output, hidden = self.rnn(embedded_with_context, hidden)

        # Predict the next token using the RNN output and the context vector
        # Combine rnn_output (squeezed) and context_vector (already squeezed)
        combined_input_for_fc = torch.cat((rnn_output.squeeze(1), context_vector), dim=1)

        prediction = self.fc_out(combined_input_for_fc) # [batch_size, output_dim]

        return prediction, hidden # Return prediction and updated hidden state

## Seq2Seq Model with GRU and Attention

class Seq2SeqGRU(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device

        # Ensure encoder and decoder have the same number of layers
        assert encoder.num_layers == decoder.num_layers, \
            "Encoder and decoder must have equal number of layers!"

    def forward(self, src, trg):
        # src: [batch_size, max_src_len, input_dim]
        # trg: [batch_size, max_tgt_len] (contains token IDs)

        batch_size = trg.shape[0]
        max_tgt_len = trg.shape[1]
        trg_vocab_size = self.decoder.output_dim

        # Tensor to store decoder outputs (logits)
        outputs = torch.zeros(batch_size, max_tgt_len, trg_vocab_size).to(self.device)

        # Encode source sentence
        # encoder_outputs: [batch_size, max_src_len, hidden_dim]
        # encoder_hidden: [num_layers, batch_size, hidden_dim]
        encoder_outputs, encoder_hidden = self.encoder(src)

        # Initialize decoder's hidden state from the encoder's final hidden state
        decoder_hidden = encoder_hidden

        # First input to the decoder is the <sos> token
        input = trg[:, 0] # [batch_size]

        # Iterate through the target sequence (excluding the <sos> token at index 0)
        for t in range(1, max_tgt_len):
            # Pass input token, hidden state, and encoder outputs to decoder
            # output: [batch_size, trg_vocab_size] (logits for next token)
            # decoder_hidden: [num_layers, batch_size, hidden_dim] (updated hidden state)
            output, decoder_hidden = self.decoder(
                input, decoder_hidden, encoder_outputs
            )

            # Store the output prediction
            outputs[:, t, :] = output

            # Decide next input token using teacher forcing
            input = trg[:, t]

        return outputs
"""

'\n# trial - Attention Module \nclass Attention(nn.Module):\n    def __init__(self, hidden_dim):\n        super().__init__()\n        # Linear layer to combine decoder hidden state and encoder outputs for \'importance\' calculation\n        # hidden_dim * 2 because we concatenate decoder_hidden_last_layer and encoder_output\n        self.attn = nn.Linear(hidden_dim * 2, hidden_dim)\n        # Linear layer to project the importance to a scalar score\n        self.v = nn.Linear(hidden_dim, 1, bias=False)\n\n    def forward(self, decoder_hidden_last_layer, encoder_outputs):\n        # decoder_hidden_last_layer: [batch_size, hidden_dim] (last layer hidden state of decoder)\n        # encoder_outputs: [batch_size, max_src_len, hidden_dim] (all encoder outputs from encoder)\n\n        batch_size = encoder_outputs.shape[0]\n        max_src_len = encoder_outputs.shape[1]\n\n        # Repeat the decoder\'s last layer hidden state \n        # This prepares it for concatenation with each encoder 

In [19]:
# trial - Attention Module
class Attention(nn.Module):
    def __init__(self, hidden_dim):
        super().__init__()
        # Linear layer to combine decoder hidden state and encoder outputs for 'importance' calculation
        # hidden_dim (decoder last layer) + hidden_dim * 2 (bidirectional encoder output)
        self.attn = nn.Linear(hidden_dim * 3, hidden_dim)  # Corrected input dimension
        # Linear layer to project the importance to a scalar score
        self.v = nn.Linear(hidden_dim, 1, bias=False)

    def forward(self, decoder_hidden_last_layer, encoder_outputs):
        # decoder_hidden_last_layer: [batch_size, hidden_dim] (last layer hidden state of decoder)
        # encoder_outputs: [batch_size, max_src_len, hidden_dim * 2] (all encoder outputs from *bidirectional* encoder)

        batch_size = encoder_outputs.shape[0]
        max_src_len = encoder_outputs.shape[1]

        # Repeat the decoder's last layer hidden state
        # This prepares it for concatenation with each encoder output
        # decoder_hidden_last_layer_repeated: [batch_size, max_src_len, hidden_dim]
        decoder_hidden_last_layer_repeated = decoder_hidden_last_layer.unsqueeze(1).repeat(1, max_src_len, 1)

        # Concatenate the repeated decoder hidden state with all encoder outputs
        # energy_input: [batch_size, max_src_len, hidden_dim + hidden_dim * 2] = [batch_size, max_src_len, hidden_dim * 3]
        energy_input = torch.cat((decoder_hidden_last_layer_repeated, encoder_outputs), dim=2)

        # Apply the attention scoring function (tanh and then linear projection)
        # energy: [batch_size, max_src_len, hidden_dim]
        energy = torch.tanh(self.attn(energy_input))

        # Project the energy to a single scalar score for each source token
        # attention_scores: [batch_size, max_src_len]
        attention_scores = self.v(energy).squeeze(2)

        # Apply softmax to get attention weights, ensuring they sum to 1 across the source sequence
        # attention_weights: [batch_size, max_src_len]
        return F.softmax(attention_scores, dim=1)


## Encoder using Bidirectional GRU (Refined)

class EncoderBiGRU(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.num_layers = num_layers
        # Use nn.GRU with bidirectional=True
        self.rnn = nn.GRU(input_dim, hidden_dim, num_layers=num_layers, bidirectional=True, batch_first=True)

        # Linear layer to project the concatenated forward/backward final hidden states
        # from hidden_dim * 2 to hidden_dim to match the decoder's hidden state size.
        self.fc_hidden = nn.Linear(hidden_dim * 2, hidden_dim)
        # Initialize this linear layer with appropriate weights (optional but can help)
        # nn.init.xavier_uniform_(self.fc_hidden.weight) # Example initialization

    def forward(self, src):
        # src: [batch_size, max_src_len, input_dim]
        # output: [batch_size, max_src_len, hidden_dim * 2]
        # hidden: [num_layers * 2, batch_size, hidden_dim]
        output, hidden = self.rnn(src)

        # Combine forward and backward final hidden states for each layer
        # hidden_forward: [num_layers, batch_size, hidden_dim]
        # hidden_backward: [num_layers, batch_size, hidden_dim]
        hidden_forward = hidden[0:self.num_layers]
        hidden_backward = hidden[self.num_layers:]

        # Concatenate forward and backward states for each layer
        # combined_hidden: [num_layers, batch_size, hidden_dim * 2]
        combined_hidden = torch.cat((hidden_forward, hidden_backward), dim=2)

        # Project the combined hidden state to the decoder's expected hidden dimension
        # final_encoder_hidden: [num_layers, batch_size, hidden_dim]
        final_encoder_hidden = torch.relu(self.fc_hidden(combined_hidden)) # Use ReLU or Tanh activation

        # Return all time-step outputs (for attention) and the processed final hidden state (for decoder initialization)
        return output, final_encoder_hidden

## Decoder using GRU with Attention

class DecoderGRU(nn.Module):
    def __init__(self, output_dim, hidden_dim, num_layers, attention):
        super().__init__()
        self.hidden_dim = hidden_dim
        self.output_dim = output_dim
        self.num_layers = num_layers
        self.attention = attention # Attention module instance

        # Embedding for target language tokens
        self.embedding = nn.Embedding(output_dim, hidden_dim)
        # Use nn.GRU instead of nn.RNN or nn.LSTM
        # Input dimension is hidden_dim (from embedding) + hidden_dim*2 (from context vector)
        self.rnn = nn.GRU(hidden_dim * 3, hidden_dim, num_layers=num_layers, batch_first=True) # <--- Change this line
        # Final output layer: takes concatenated RNN output (hidden_dim) and context vector (hidden_dim*2)
        self.fc_out = nn.Linear(hidden_dim * 3, output_dim) # <--- Change this line

    def forward(self, input, hidden, encoder_outputs):
        # input: [batch_size] (a batch of single token IDs at a time step)
        # hidden: [num_layers, batch_size, hidden_dim] (initial hidden state from encoder or previous decoder step)
        # encoder_outputs: [batch_size, max_src_len, hidden_dim * 2] (all encoder outputs from *bidirectional* encoder)

        # Add a sequence dimension to the input for GRU
        input = input.unsqueeze(1) # [batch_size, 1]

        # Embed the input token
        embedded = self.embedding(input) # [batch_size, 1, hidden_dim]

        # Get the hidden state of the last layer of the decoder for attention calculation
        # decoder_hidden_last_layer: [batch_size, hidden_dim]
        decoder_hidden_last_layer = hidden[-1]

        # Calculate attention weights based on decoder's current hidden state and all encoder outputs
        # attention_weights: [batch_size, max_src_len]
        attention_weights = self.attention(decoder_hidden_last_layer, encoder_outputs)

        # Apply attention weights to encoder outputs to get the context vector
        # context_vector: [batch_size, hidden_dim * 2]
        context_vector = torch.bmm(attention_weights.unsqueeze(1), encoder_outputs).squeeze(1)

        # Concatenate the embedded input with the context vector for the GRU input
        # embedded_with_context: [batch_size, 1, hidden_dim + hidden_dim * 2] = [batch_size, 1, hidden_dim * 3]
        embedded_with_context = torch.cat((embedded, context_vector.unsqueeze(1)), dim=2)

        # Pass combined input and previous hidden state through GRU
        # rnn_output: [batch_size, 1, hidden_dim] (output of the top GRU layer)
        # hidden: [num_layers, batch_size, hidden_dim] (updated hidden state)
        rnn_output, hidden = self.rnn(embedded_with_context, hidden)

        # Predict the next token using the RNN output and the context vector
        # Combine rnn_output (squeezed, hidden_dim) and context_vector (already squeezed, hidden_dim * 2)
        # combined_input_for_fc: [batch_size, hidden_dim + hidden_dim * 2] = [batch_size, hidden_dim * 3]
        combined_input_for_fc = torch.cat((rnn_output.squeeze(1), context_vector), dim=1)

        prediction = self.fc_out(combined_input_for_fc) # [batch_size, output_dim]

        return prediction, hidden # Return prediction and updated hidden state


## Seq2Seq Model with GRU and Attention

class Seq2SeqGRU(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.device = device

        # Ensure encoder and decoder have the same number of layers
        assert encoder.num_layers == decoder.num_layers, \
            "Encoder and decoder must have equal number of layers!"

    def forward(self, src, trg):
        # src: [batch_size, max_src_len, input_dim]
        # trg: [batch_size, max_tgt_len] (contains token IDs)

        batch_size = trg.shape[0]
        max_tgt_len = trg.shape[1]
        trg_vocab_size = self.decoder.output_dim

        # Tensor to store decoder outputs (logits)
        outputs = torch.zeros(batch_size, max_tgt_len, trg_vocab_size).to(self.device)

        # Encode source sentence
        # encoder_outputs: [batch_size, max_src_len, hidden_dim]
        # encoder_hidden: [num_layers, batch_size, hidden_dim]
        encoder_outputs, encoder_hidden = self.encoder(src)

        # Initialize decoder's hidden state from the encoder's final hidden state
        decoder_hidden = encoder_hidden

        # First input to the decoder is the <sos> token
        input = trg[:, 0] # [batch_size]

        # Iterate through the target sequence (excluding the <sos> token at index 0)
        for t in range(1, max_tgt_len):
            # Pass input token, hidden state, and encoder outputs to decoder
            # output: [batch_size, trg_vocab_size] (logits for next token)
            # decoder_hidden: [num_layers, batch_size, hidden_dim] (updated hidden state)
            output, decoder_hidden = self.decoder(
                input, decoder_hidden, encoder_outputs
            )

            # Store the output prediction
            outputs[:, t, :] = output

            # Decide next input token using teacher forcing
            input = trg[:, t]

        return outputs

In [21]:

# Define model parameters
INPUT_DIM = 100  # GloVe embedding dimension
OUTPUT_DIM = hin_vocab_size  # IndicBERT vocabulary size
HIDDEN_DIM = 256
NUM_LAYERS = 2 # Set number of layers for both encoder and decoder

# Instantiate the Attention mechanism
attention = Attention(HIDDEN_DIM).to(device)
'''

# Instantiate the GRU-based Encoder and Decoder
encoder = EncoderGRU(INPUT_DIM, HIDDEN_DIM, NUM_LAYERS).to(device)
decoder = DecoderGRU(OUTPUT_DIM, HIDDEN_DIM, NUM_LAYERS, attention).to(device)

# Instantiate the Seq2Seq model with GRU components
model = Seq2SeqGRU(encoder, decoder, device).to(device)
'''


# Instantiate the Bidirectional GRU-based Encoder
encoder = EncoderBiGRU(INPUT_DIM, HIDDEN_DIM, NUM_LAYERS).to(device)
# Instantiate the GRU-based Decoder (which expects a hidden_dim matching the encoder's output after projection)
decoder = DecoderGRU(OUTPUT_DIM, HIDDEN_DIM, NUM_LAYERS, attention).to(device)

# Instantiate the Seq2Seq model with GRU components
# Ensure the Seq2Seq model passes encoder_outputs and encoder_hidden correctly to the decoder
model = Seq2SeqGRU(encoder, decoder, device).to(device)

# Define loss function and optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
# We ignore the padding token's contribution to the loss
PAD_IDX = tokenizer_hin.pad_token_id
criterion = nn.CrossEntropyLoss(ignore_index=PAD_IDX)

# Training loop
N_EPOCHS = 5

for epoch in range(N_EPOCHS):
    model.train()
    epoch_loss = 0

    for i, (src, trg) in enumerate(train_loader):
        if i % 500 == 0:
            print(f"Batch {i}/{len(train_loader)}")
        src = src.to(device)
        trg = trg.to(device)

        optimizer.zero_grad()

        output = model(src, trg)

        # output: [batch_size, max_tgt_len, output_dim]
        # trg: [batch_size, max_tgt_len]

        # Reshape for CrossEntropyLoss
        output = output[:, 1:].reshape(-1, output.shape[-1]) # Ignore <sos> token prediction
        trg = trg[:, 1:].reshape(-1) # Ignore <sos> token ground truth

        loss = criterion(output, trg)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    print(f'Epoch: {epoch+1:02} | Train Loss: {epoch_loss / len(train_loader):.3f}')

    # Validation loop
    model.eval()
    val_loss = 0
    with torch.no_grad():
        for i, (src, trg) in enumerate(val_loader):
            src = src.to(device)
            trg = trg.to(device)

            output = model(src, trg)

            output = output[:, 1:].reshape(-1, output.shape[-1])
            trg = trg[:, 1:].reshape(-1)

            loss = criterion(output, trg)
            val_loss += loss.item()

    print(f'Epoch: {epoch+1:02} | Val Loss: {val_loss / len(val_loader):.3f}')


'''
def calculate_bleu(data_loader, model, max_src_len, max_tgt_len, device, tokenizer_hin):
    trgs = []
    pred_trgs = []

    model.eval()

    with torch.no_grad():
        for i, (src, trg) in enumerate(data_loader):
            # We need the original target sentences as strings for BLEU, not the padded tensors
            # The DataLoader provides the padded tensors, so we need to get the original
            # sentences corresponding to this batch from the validation dataset.
            # This is not ideal for large datasets, but for demonstration, we can
            # retrieve the original sentences. A better approach would be to store
            # original sentences in the dataset or pass them alongside tensors.

            # For this example, we'll translate sentences one by one from the validation dataset
            # based on the batch index, which is inefficient but works for demonstration.
            # In a real scenario, process batches more efficiently.

            batch_original_src = [source_sentences_val[j + i * data_loader.batch_size] for j in range(src.shape[0])]
            batch_original_trg = [target_sentences_val[j + i * data_loader.batch_size] for j in range(trg.shape[0])]


            for j in range(len(batch_original_src)):
                src_sentence = batch_original_src[j]
                trg_sentence = batch_original_trg[j]

                # Tokenize reference sentence for BLEU
                # IndicBERT tokenizer might produce sub-words, BLEU works best on word tokens
                # Let's try tokenizing the reference using a simple split or a more robust tokenizer if needed
                # For simplicity, we will use the IndicBERT tokenizer output for both prediction and reference
                # This might slightly lower the BLEU score compared to using word tokens for reference.
                # A more standard approach is to tokenize reference by spaces.
                reference_tokens = tokenizer_hin(trg_sentence, truncation=False, padding=False)["input_ids"]
                # Remove special tokens from reference for BLEU calculation
                reference_tokens_filtered = [tok for tok in reference_tokens if tok not in tokenizer_hin.all_special_ids]
                reference = [tokenizer_hin.decode(reference_tokens_filtered).split()] # BLEU expects list of reference translations

                # Translate the source sentence
                translated_sentence_str = translate_sentence(model, src_sentence, max_src_len, max_tgt_len, device, tokenizer_hin)
                # Tokenize the predicted sentence for BLEU
                prediction = translated_sentence_str.split()

                trgs.append(reference)
                pred_trgs.append(prediction)

    # Calculate BLEU score
    # SmoothingFunction is used to handle cases where n-grams are not present in the reference
    smoothie = SmoothingFunction().method4
    bleu_score = sentence_bleu(trgs, pred_trgs, smoothing_function=smoothie)

    return bleu_score
'''

Batch 0/1719
Batch 500/1719
Batch 1000/1719
Batch 1500/1719
Epoch: 01 | Train Loss: 6.038
Epoch: 01 | Val Loss: 5.225
Batch 0/1719
Batch 500/1719
Batch 1000/1719
Batch 1500/1719
Epoch: 02 | Train Loss: 4.546
Epoch: 02 | Val Loss: 4.799
Batch 0/1719
Batch 500/1719
Batch 1000/1719
Batch 1500/1719
Epoch: 03 | Train Loss: 3.512
Epoch: 03 | Val Loss: 4.767
Batch 0/1719
Batch 500/1719
Batch 1000/1719
Batch 1500/1719
Epoch: 04 | Train Loss: 2.604
Epoch: 04 | Val Loss: 4.954
Batch 0/1719
Batch 500/1719
Batch 1000/1719
Batch 1500/1719
Epoch: 05 | Train Loss: 2.011
Epoch: 05 | Val Loss: 5.175


'\ndef calculate_bleu(data_loader, model, max_src_len, max_tgt_len, device, tokenizer_hin):\n    trgs = []\n    pred_trgs = []\n\n    model.eval()\n\n    with torch.no_grad():\n        for i, (src, trg) in enumerate(data_loader):\n            # We need the original target sentences as strings for BLEU, not the padded tensors\n            # The DataLoader provides the padded tensors, so we need to get the original\n            # sentences corresponding to this batch from the validation dataset.\n            # This is not ideal for large datasets, but for demonstration, we can\n            # retrieve the original sentences. A better approach would be to store\n            # original sentences in the dataset or pass them alongside tensors.\n\n            # For this example, we\'ll translate sentences one by one from the validation dataset\n            # based on the batch index, which is inefficient but works for demonstration.\n            # In a real scenario, process batches more effic

In [ ]:
# debugger = 0
# Evaluation with BLEU score
'''
def translate_sentence(model, sentence, max_src_len, max_tgt_len, device, tokenizer_hin):

    model.eval()
    with torch.no_grad():
        # Encode source sentence
        src_vec = encode_glove(sentence, max_src_len).unsqueeze(0).to(device) # Add batch dimension
        encoder_hidden = model.encoder(src_vec)

        # Initialize decoder input with <sos> token
        # For IndicBERT, the CLS token (usually 101) serves as <sos>
        input_token = torch.tensor([tokenizer_hin.cls_token_id], device=device) # [1]

        translated_tokens = []

        for _ in range(max_tgt_len -1): # -1 because we already have the first token
            # Pass input token and hidden state to decoder
            output, encoder_hidden = model.decoder(input_token, encoder_hidden)

            # Get the most probable next token
            pred_token = output.argmax(1) # [1]

            # Add predicted token to list (excluding <sos> if it's predicted later)
            if pred_token.item() != tokenizer_hin.cls_token_id:
                 translated_tokens.append(pred_token.item())

            # Stop if <eos> token is predicted
            if pred_token.item() == tokenizer_hin.sep_token_id:
                break

            # Use the predicted token as the next input
            input_token = pred_token

        # Decode the translated tokens
        translated_sentence = tokenizer_hin.decode(translated_tokens, skip_special_tokens=True)

        while debugger < 10:
            print(f"\nTranslating sentence: '{sentence}'")
            print(f"Source tensor shape: {src_vec.shape}")
            print(f"Encoder final hidden state shape: {encoder_hidden.shape}")
            print(f"Translated sentence: '{translated_sentence}'")
            debugger += 1

        return translated_sentence'''

'\ndef translate_sentence(model, sentence, max_src_len, max_tgt_len, device, tokenizer_hin):\n\n    model.eval()\n    with torch.no_grad():\n        # Encode source sentence\n        src_vec = encode_glove(sentence, max_src_len).unsqueeze(0).to(device) # Add batch dimension\n        encoder_hidden = model.encoder(src_vec)\n\n        # Initialize decoder input with <sos> token\n        # For IndicBERT, the CLS token (usually 101) serves as <sos>\n        input_token = torch.tensor([tokenizer_hin.cls_token_id], device=device) # [1]\n\n        translated_tokens = []\n\n        for _ in range(max_tgt_len -1): # -1 because we already have the first token\n            # Pass input token and hidden state to decoder\n            output, encoder_hidden = model.decoder(input_token, encoder_hidden)\n\n            # Get the most probable next token\n            pred_token = output.argmax(1) # [1]\n\n            # Add predicted token to list (excluding <sos> if it\'s predicted later)\n            i

In [22]:
## Saving Model Weights

# Define a path to save the model
MODEL_SAVE_PATH = 'seq2seq_gru_attention_bi_model.pth'

print(f"Saving model state to {MODEL_SAVE_PATH}...")
# Save the model's state dictionary
torch.save(model.state_dict(), MODEL_SAVE_PATH)
print("Model state saved successfully.")

Saving model state to seq2seq_gru_attention_bi_model.pth...
Model state saved successfully.


In [23]:

# Evaluation with BLEU score (modified for attention)
def translate_sentence(model, sentence, max_src_len, max_tgt_len, device, tokenizer_hin):
    model.eval()
    with torch.no_grad():
        # Encode source sentence
        # The encoder returns (outputs, hidden) for nn.RNN
        encoder_outputs, encoder_hidden = model.encoder(encode_glove(sentence, max_src_len).unsqueeze(0).to(device))

        # The hidden state returned by the encoder is [num_layers, batch_size, hidden_dim]
        # We use this directly as the initial hidden state for the decoder.
        # If the decoder was an LSTM/GRU, we would need to handle the cell state too,
        # but since it's also an RNN, the hidden state is sufficient.

        input_token = torch.tensor([tokenizer_hin.cls_token_id], device=device)
        translated_tokens = []

        # Initialize decoder's hidden state directly from the encoder's final hidden state
        decoder_hidden = encoder_hidden

        for _ in range(max_tgt_len - 1):
            # Decoder now expects hidden state (not a tuple) and encoder_outputs
            output, decoder_hidden = model.decoder(input_token, decoder_hidden, encoder_outputs)
            pred_token = output.argmax(1)

            if pred_token.item() != tokenizer_hin.cls_token_id:
                translated_tokens.append(pred_token.item())

            if pred_token.item() == tokenizer_hin.sep_token_id:
                break

            input_token = pred_token

        translated_sentence = tokenizer_hin.decode(translated_tokens, skip_special_tokens=True)
        return translated_sentence

In [24]:

def calculate_bleu(data_loader, model, max_src_len, max_tgt_len, device, tokenizer_hin):
    trgs = []
    pred_trgs = []

    model.eval()

    with torch.no_grad():
        for i, (src, trg) in enumerate(data_loader):
            # We need the original target sentences as strings for BLEU, not the padded tensors
            # The DataLoader provides the padded tensors, so we need to get the original
            # sentences corresponding to this batch from the validation dataset.
            # This is not ideal for large datasets, but for demonstration, we can
            # retrieve the original sentences. A better approach would be to store
            # original sentences in the dataset or pass them alongside tensors.

            # For this example, we'll translate sentences one by one from the validation dataset
            # based on the batch index, which is inefficient but works for demonstration.
            # In a real scenario, process batches more efficiently.

            # When evaluating on the test set, you should retrieve original sentences
            # from the test dataset lists, not the validation ones.
            batch_original_src = [source_sentences_test[j + i * data_loader.batch_size] for j in range(src.shape[0])]
            batch_original_trg = [target_sentences_test[j + i * data_loader.batch_size] for j in range(trg.shape[0])]


            for j in range(len(batch_original_src)):
                src_sentence = batch_original_src[j]
                trg_sentence = batch_original_trg[j]

                # Tokenize reference sentence for BLEU
                # IndicBERT tokenizer might produce sub-words, BLEU works best on word tokens
                # Let's try tokenizing the reference using a simple split or a more robust tokenizer if needed
                # For simplicity, we will use the IndicBERT tokenizer output for both prediction and reference
                # This might slightly lower the BLEU score compared to using word tokens for reference.
                # A more standard approach is to tokenize reference by spaces.
                reference_tokens = tokenizer_hin(trg_sentence, truncation=False, padding=False)["input_ids"]
                # Remove special tokens from reference for BLEU calculation
                reference_tokens_filtered = [tok for tok in reference_tokens if tok not in tokenizer_hin.all_special_ids]
                # Decode and split the filtered tokens into words
                tokenized_reference = tokenizer_hin.decode(reference_tokens_filtered).split()

                # Translate the source sentence
                translated_sentence_str = translate_sentence(model, src_sentence, max_src_len, max_tgt_len, device, tokenizer_hin)
                # Tokenize the predicted sentence for BLEU
                prediction = translated_sentence_str.split()

                # Append the reference as a list containing ONE list of tokens
                trgs.append([tokenized_reference]) # BLEU expects list of reference translations for one candidate
                # Append the prediction as a list of tokens
                pred_trgs.append(prediction)

    # Calculate BLEU score
    # SmoothingFunction is used to handle cases where n-grams are not present in the reference
    smoothie = SmoothingFunction().method4
    # Use corpus_bleu for calculating BLEU over multiple sentences
    from nltk.translate.bleu_score import corpus_bleu
    bleu_score = corpus_bleu(trgs, pred_trgs, smoothing_function=smoothie)

    return bleu_score

# Load test data
df_test = pd.read_csv("/content/translation_engtohin_test.csv").dropna()
source_sentences_test = df_test["source"].tolist()
target_sentences_test = df_test["target"].tolist()

test_dataset = TranslationDataset(source_sentences_test, target_sentences_test, max_src_len, max_tgt_len)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_fn)

# Calculate BLEU on the test set
print("Calculating BLEU score on the test set...")
test_bleu_score = calculate_bleu(test_loader, model, max_src_len, max_tgt_len, device, tokenizer_hin)
print(f"Test BLEU score: {test_bleu_score:.4f}")


Calculating BLEU score on the test set...
Test BLEU score: 0.0092


Reading the Test Data

In [ ]:
import pandas as pd
# Load the test dataset
df = pd.read_csv('translation_engtohin_test.csv')
print(df.head())

                                              source  \
0  All these three centres, incidentally, are the...   
1                 (For the full story, click here ).   
2  He was also questioned whether action will be ...   
3                    Students should not be worried.   
4  Ridiculing BJP leaders, Yadav said when he was...   

                                              target  
0  दिलचस्प बात है कि ये तीनों केंद्र बीसीसीआई अध्...  
1         (पूरा एपिसोड देखने के लिए यहां क्लिक करें)  
2  उनसे भाजपा नेता कपिल मिश्रा के खिलाफ कार्रवाई ...  
3                ' छात्रों को निराश नहीं किया जाएगा.  
4  यादव ने कहा, 'रक्षा मंत्री रहते मैं खुद भारत-प...  


Addition information:

* Our suggestion is to use Google Colab’s free T4 GPU for training the RNN model. You can refer to this video: https://www.youtube.com/watch?v=GooQPDMH2Uk for guidance. To use the GPU, you should set the device as 'cuda' using:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu').
Then, make sure to move all model components and data tensors to this device using .to(device) before training and evaluation.
* To save model weights use: "torch.save(model.state_dict(), "translation_model.pt")"
* Pick any 5 sentences, print the source, target and predicted sentences for them.  
* The code should be flexible such that the dimension of the input character embeddings, the hidden states of the encoders
 and decoders, and the number of layers in the encoder and decoder can be changed.
* Mention the loss, optimizer used, learning rate and the number of epochs trained for.
* Report the training loss, accuracy and validation loss,accuracy plots.
* Use Bleu Score to evaluate your models performance
* You are free to make any other changes in the code as you require

## Referencees

https://nlp.stanford.edu/pubs/glove.pdf
